In [166]:
import petl as etl
import pandas as pd

from urllib.parse import urljoin, urlencode

from pathlib import Path

In [167]:
column_mapper = {
    'OBS_VALUE': 'value',

    'GEOGRAPHY_CODE': 'geography_code',
    'GEOGRAPHY_NAME': 'geography_name',

    'C_SEX_TYPE': 'variable',
    'C_SEX_NAME': 'category',
    'C_SEX_SORTORDER': 'sortorder',

    'C2021_AGE_12A_TYPE': 'variable',
    'C2021_AGE_12A_NAME': 'category',
    'C2021_AGE_12A_SORTORDER': 'sortorder',

    'C2021_ETH_20_TYPE': 'variable',
    'C2021_ETH_20_NAME': 'category',
    'C2021_ETH_20_SORTORDER': 'sortorder',

    'MEASURES_NAME': 'measure',
}

In [168]:
geography='633350022...633350024,633350065,633350064,633350066...633350072,633350139...633350141,633350143,633350142,633350144...633350146,633350079...633350082,633350092,633350100...633350102,633350073...633350078,633350083,633350085,633350093,633350094,633350096...633350098,633349955,633349956,633350136...633350138,633350084,633350086,633350087,633350089,633350088,633350090,633350091,633350099,633350227,633349949...633349951,633349953,633350095,633350228,633350230,633350231,633349954,633349957,633349958,633349960,633349961,633349944,633349945,633349947,633349948,633349952,633350056...633350059,633349946,633349959,633350155...633350157,633350161,633350053...633350055,633350060...633350063,633350153,633350158,633350159,633371653,633371658,633350148,633350149,633350151,633350152,633350154,633349962...633349965,633349967,633349969,633349972,633350222...633350226,633350229,633349979...633349981,633350147,633350150,633350160,633350162...633350164,633350028,633350029,633350031...633350033,633350025,633350026,633350030,633350196,633349973...633349976,633349978,633350044,633350045,633350047,633350049,633350043,633350048,633350176,633371844,633371845,633349966,633349968,633349970,633349971,633349977,633350027,633350195,633350197,633350175,633350177...633350180,633350050,633350052,633350194,633350209,633350210,633350198...633350202,633350046,633350051,633350170...633350174,633350181,633350182,633350184,633349994,633349995,633350001...633350008,633350034,633350183,633350203,633350212,633349996...633350000,633350165...633350169,633350204...633350208,633371846,633371847,633349984,633349989,633350103,633350107,633349985...633349987,633349990,633350009...633350013,633350104...633350106,633350109,633350110,633350035,633350036,633350038,633350133,633371659...633371661,633350037,633350039...633350041,633350108,633350111...633350113,633350117,633349988,633350190...633350193,633349991,633349992,633350115,633350116,633350215,633350216,633350220,633350221,633350042,633350128,633350134,633350135,633350214,633350118,633350120...633350122,633350185...633350189,633350125...633350127,633350129...633350132,633350114,633350213,633350217...633350219,633350232,633350235,633350242,633350119,633350123,633350124,633350233,633350234,633350236...633350241,633350014,633350018,633350021,633350020,633350019,633350016,633350017,633350015,633371657,633349993,633349983,633371656,633349982,633350211,633371655,633371654,641728941...641728970'

In [169]:
def census_query(url, **params):
    return '?'.join([
        url,
        urlencode({
            'date': 'latest',
            'geography': geography,
            'measures': '20100,20301',
            **params,
        }, safe=",")
    ])

In [170]:
sex_query = census_query('https://www.nomisweb.co.uk/api/v01/dataset/NM_2028_1.data.csv', c_sex='1,2')
age_query = census_query('https://www.nomisweb.co.uk/api/v01/dataset/NM_2018_1.data.csv', c2021_age_12a='1...11')
eth_query = census_query('https://www.nomisweb.co.uk/api/v01/dataset/NM_2041_1.data.csv', c2021_eth_20='1001...1005')

In [171]:
def process(table):
    return (
        table
        # Filter only status 'Normal Value' and conf 'Free'
        .select(lambda r: r.OBS_STATUS == 'A' and r.OBS_CONF == 'F')
        # Rename fields
        .rename(column_mapper, strict=False)
        # Keep only fields of interest
        .cut(
            'geography_code',
            'geography_name',
            'variable',
            'category',
            'sortorder',
            'measure',
            'value',
        )
        # Cache for performance
        .cache()
    )

In [172]:
data = etl.cat(
    process(etl.fromcsv(sex_query)),
    process(etl.fromcsv(age_query)),
    process(etl.fromcsv(eth_query)),
).convertnumbers()

In [173]:
DATA = Path('data')
DATA.mkdir(exist_ok=True)

In [174]:
data.cut('variable', 'category', 'sortorder').distinct().sort(['variable', 'sortorder']).tocsv(DATA / 'census_categories.csv')

In [175]:
def make_index(df):
    df.geography_code = pd.CategoricalIndex(df.geography_code)
    df.geography_name = pd.CategoricalIndex(df.geography_name)
    df.variable = pd.CategoricalIndex(df.variable)
    df.category = pd.CategoricalIndex(df.category)    
    df.measure = pd.CategoricalIndex(df.measure)
    return df
# .set_index(['geography_code', 'geography_name', 'variable', 'category', 'sortorder', 'measure'])

In [176]:
data.cutout('sortorder').todataframe().pipe(make_index).to_parquet(DATA / 'census.parquet')